In [ ]:
# 비용 함수 구현

import torch
import torch.nn.functional as F

torch.manual_seed(1)

# 로우 레벨
# 첫번째 수식
# (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
# 두번째 수식
# (y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()


# 하이 레벨
# 첫 번째 수식
# F.nll_loss(F.log_softmax(z, dim=1), y)
# 두 번째 수식
# F.cross_entropy(z, y) // 비용 함수에 소프트맥스 함수까지 포함
# 세 번째 수식
# nn.CrossEntropyLoss() 클래스 사용
# cost = nn.CrossEntropyLoss()(z, y)

----------------------------------------------------------------------------------------------------

# 1단계: 클래스로 객체 생성
criterion = nn.CrossEntropyLoss()

# 2단계: 생성된 객체 사용
loss = criterion(z, y)
# 같은 객체로 여러 번 계산 가능
loss1 = criterion(z, y)           # 첫 번째 계산
loss2 = criterion(z, y)           # 두 번째 계산

# 새로운 데이터가 있다면
z2 = torch.rand(3, 5, requires_grad=True)
y2 = torch.randint(5, (3,)).long()
loss3 = criterion(z2, y2)         # 새 데이터로 계산


# 같은 객체로 여러 번 계산 가능
loss1 = criterion(z, y)           # 첫 번째 계산
loss2 = criterion(z, y)           # 두 번째 계산

# 새로운 데이터가 있다면
z2 = torch.rand(3, 5, requires_grad=True)
y2 = torch.randint(5, (3,)).long()
loss3 = criterion(z2, y2)         # 새 데이터로 계산

----------------------------------------------------------------------------------------------------

# 평균 대신 합계를 구하는 설정
# 손실함수 객체를 한 번만 생성. 이제 호출할때는 무조건 criterion으로만 호출함.
criterion = nn.CrossEntropyLoss(reduction='sum')

# 같은 객체로 여러 번 계산 가능
loss1 = criterion(z, y)           # 첫 번째 계산
loss2 = criterion(z, y)           # 두 번째 계산

# 새로운 데이터가 있다면
z2 = torch.rand(3, 5, requires_grad=True)
y2 = torch.randint(5, (3,)).long()
loss3 = criterion(z2, y2)         # 새 데이터로 계산

----------------------------------------------------------------------------------------------------

# 함수 방식에서는 매번 설정을 반복해야 함
loss_sum1 = F.cross_entropy(z, y, reduction='sum')
loss_sum2 = F.cross_entropy(z2, y2, reduction='sum')  # 설정 반복


In [ ]:
# 로우-레벨 구현

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

W = torch.randn((4, 3), requires_grad=True)
b = torch.randn((1, 3), requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [ ]:
# 하이-레벨 구현

W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


In [ ]:
# nn.Module 사용

# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [ ]:
# 클래스 사용

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [ ]:
# MNIST 데이터 분류

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# hyperparameters
training_epochs = 15
batch_size = 100

# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

# dataset loader
data_loader = DataLoader(dataset=mnist_train,
                                          batch_size=batch_size, # 배치 크기는 100
                                          shuffle=True,
                                          drop_last=True)

# MNIST data image of shape 28 * 28 = 784
linear = nn.Linear(784, 10, bias=True).to(device)

criterion = nn.CrossEntropyLoss().to(device) # 비용 함수 객체
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1) # 최적화 객체

In [ ]:
for epoch in range(training_epochs): # 앞서 training_epochs의 값은 15로 지정함.
    avg_cost = 0
    total_batch = len(data_loader)

    for X, Y in data_loader:
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28 * 28).to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')


In [ ]:
# 테스트 데이터를 사용하여 모델을 테스트한다.
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # MNIST 테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

    plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()
